# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import os
import joblib
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-136763
aml-quickstarts-136763
southcentralus
61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
#create compute cluster
amlcompute_cluster_name="aml"

#verify that cluster does not already exist
try:
    aml_compute=ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config=AmlCompute.provisioning_configuration(vm_size="Standard_D3_V2",max_nodes=4)
    aml_compute=ComputeTarget.create(ws,amlcompute_cluster_name,compute_config)

aml_compute.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [3]:
# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
auto_ds_path="https://raw.githubusercontent.com/shat700/nd00333-capstone/master/starter_file/heart_failure_clinical_records_dataset.csv"
ds=TabularDatasetFactory.from_delimited_files(path=auto_ds_path)

#preview data
df = ds.to_pandas_dataframe()
df.describe()



,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [4]:
import logging
import time

# TODO: Put your automl settings here
automl_settings = {
    "n_cross_validations":5,
    "primary_metric":'accuracy',
    "enable_early_stopping":True,
    "experiment_timeout_minutes":15,
    "max_concurrent_iterations":10,
   }

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task='classification',
    compute_target='aml',
    training_data=ds,
    label_column_name='DEATH_EVENT',
    **automl_settings
    )

In [5]:
from azureml.core.experiment import Experiment
# choose a name for experiment
experiment_name = 'automl_run'
experiment=Experiment(ws, experiment_name)

# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on aml with default configuration
Running on remote compute: aml
Parent Run ID: AutoML_2390dc8f-b818-486e-94de-b3e3bf41227d

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in t

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
# Retrieve and save your best automl model.
best_remote_run,best_automl_model=remote_run.get_output()
best_run_metrics=remote_run.get_metrics()
for primary_metric in best_run_metrics:
    metric=best_run_metrics[primary_metric]
    print(primary_metric,metric)


experiment_status ['DatasetEvaluation', 'FeaturesGeneration', 'DatasetFeaturization', 'DatasetFeaturizationCompleted', 'DatasetCrossValidationSplit', 'ModelSelection']
experiment_status_description ['Gathering dataset statistics.', 'Generating features for the dataset.', 'Beginning to fit featurizers and featurize the dataset.', 'Completed fit featurizers and featurizing the dataset.', 'Generating individually featurized CV splits.', 'Beginning model selection.']
f1_score_micro 0.8796610169491526
precision_score_macro 0.8919036703986553
norm_macro_recall 0.6933333333333334
precision_score_weighted 0.8966259388449416
AUC_weighted 0.9168311415651532
balanced_accuracy 0.8466666666666667
f1_score_weighted 0.8744137742963474
f1_score_macro 0.8533586706251939
accuracy 0.8796610169491526
recall_score_macro 0.8466666666666667
log_loss 0.3719978729441224
weighted_accuracy 0.9017121064304477
AUC_macro 0.9168311415651532
AUC_micro 0.9163153308436275
average_precision_score_weighted 0.925567766560

In [11]:
print(best_remote_run)
best_remote_run

Run(Experiment: automl_run,
Id: AutoML_2390dc8f-b818-486e-94de-b3e3bf41227d_47,
Type: azureml.scriptrun,
Status: Completed)


Experiment,Id,Type,Status,Details Page,Docs Page
automl_run,AutoML_2390dc8f-b818-486e-94de-b3e3bf41227d_47,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [23]:
#TODO: Save the best model
import pickle
best_remote_run,best_automl_model=remote_run.get_output()
#joblib.dump(value=best_automl_model,filename='automl_model.joblib')
pkl_filename = "automl_model.pkl"
with open(pkl_filename,'wb') as f:
    pickle.dump(best_automl_model,f)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [2]:
from azureml.core.model import Model
# Register model
model = Model.register(workspace = ws,
                        model_name = "Heart_failure_prediction",
                        model_path = "./automl_model.pkl",
                        description = "Best AutoML Model",)



Registering model Heart_failure_prediction


In [8]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies


#defining an inference configuration
service_name = 'heart-failure-automl'

#setting up a curated environment
env = Environment.get(ws, name="AzureML-AutoML")

#combining scoring script and environment in inference configuration
inference_config = InferenceConfig(entry_script='scoring.py', environment=env)

#deploy the model as a webservice
from azureml.core.webservice import LocalWebservice, Webservice

#set deployment configuration
local_deployment_config = LocalWebservice.deploy_configuration(port=8000)
service = Model.deploy(ws, "heart-failure-automl", [model], inference_config, local_deployment_config)

print(service.state)

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, enable_app_insights=True, auth_enabled=True) #service.update

service = Model.deploy(workspace=ws,
                        name='heart-failure-automl',
                        models=[model],
                        inference_config=inference_config,
                        deployment_config=aci_config,
                        overwrite=True)
service.wait_for_deployment(show_output=True)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry viennaglobal.azurecr.io
Logging into Docker registry viennaglobal.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM viennaglobal.azurecr.io/azureml/azureml_1eed3e6a679bbc599bfa3e074a25301a
 ---> 59311b131e1a
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 0c0ba4332988
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjYxYzVjM2YwLTZkYzctNGVkOS1hN2YzLWM3MDRiMjBlM2IzMCIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTEzNjc2MyIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTEzNjc2MyIsIndvcmtzcGFjZUlkIjoiN2ZiMDA2OWUtOGVjYy00ZjViLWIyNzktYjk2NDkzM2Q4MzZmIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 0f406ad86942
 ---> 789bd08c3bd7
Step 4/5 : RUN mv '/var/azureml-app/tmphwhntrjv.py' /var/azureml-app/main.py
 ---> Running in ad95a096ee22
 ---> 3d090bac59c7
Step 5/5 : 

In [9]:
#retrieve deployed model uri
print(service.scoring_uri)
print(service.swagger_uri)

http://b7300633-2535-49d8-8fa2-b7f6e10f0fb2.southcentralus.azurecontainer.io/score
http://b7300633-2535-49d8-8fa2-b7f6e10f0fb2.southcentralus.azurecontainer.io/swagger.json


In [10]:
#retrive primary key
primary, secondary= service.get_keys()
print(primary)

8kbuKEuxxegpkAGZWtwaTKxVw7DzfGCc


TODO: In the cell below, send a request to the web service you deployed to test it.

In [17]:
#Testing of the deployed webservice
import requests
import json

# URL for the web service
scoring_uri = 'http://b7300633-2535-49d8-8fa2-b7f6e10f0fb2.southcentralus.azurecontainer.io/score'
# If the service is authenticated, set the key or token
key = '8kbuKEuxxegpkAGZWtwaTKxVw7DzfGCc'

# Two sets of data to score, so we get two results back
data = {"data":
        [
            [
               55,0,7861,0,38,0,263358.03,1.1,136,1,0,6],
            [
                60,0,68,0,20,0,119000,2.9,127,1,1,64]
        ]
        }
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {primary}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

{}


TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
#logs of the webservice
logs=service.get_logs()
print(logs)

In [ ]:
#delete the deployed webservice
service.delete()